In [1]:
import os
import zipfile
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load url content into soup
url = 'https://s3.amazonaws.com/tripdata/'

r = requests.get(url)
soup = BeautifulSoup(r.text, 'xml')

In [ ]:
# extract file names from soup
files = soup.find_all('Key')
clean_files = []
for i in range(len(files)-1):
    clean_files.append(files[i].get_text())

In [ ]:
# download & unzip data files
for file in clean_files:
    file_url = url + file
    with open(file, "wb") as f:
            response = requests.get(file_url)
            f.write(response.content)
    # unzip data files
    with zipfile.ZipFile(file, "r") as zip_ref:
        zip_ref.extractall("tripdata")        
    # remove zipped file after unziping
    os.remove(file)
os.rmdir('tripdata/__MACOSX')

In [ ]:
# rename files 
directory = 'tripdata/'
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.csv'):
        new_filename = filename.replace(' ','').lower().split('ci', 1)[0].strip('-').replace('-','_')
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename + '.csv'))

In [2]:
#load into dfs (use when files have alredy been downloaded & renamed)
directory = 'tripdata/'
dfs = {}

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.csv'):
        dfs[filename.split('.')[0]] = pd.read_csv(os.path.join(directory, filename)) 

In [3]:
#make column names consistent across all dfs
col_names = ['trip_duration', 'start_time', 'stop_time', 'start_station_id', 'start_station_name', 'start_station_lat',
             'start_station_long', 'end_station_id', 'end_station_name', 'end_station_lat', 'end_station_long',
             'bike_id', 'user_type', 'birth_year', 'gender']

for df in dfs.values():
    df.columns = col_names

In [4]:
# iterate over dfs changing dates to datetime
for df in dfs.values():
    df.start_time =  pd.to_datetime(df.start_time, format='%m/%d/%Y %H:%M:%S', errors='ignore')
    df.start_time =  pd.to_datetime(df.start_time, format='%m/%d/%Y %H:%M', errors='ignore')
    df.start_time =  pd.to_datetime(df.start_time, format='%Y-%m-%d %H:%M:%S', errors='ignore')
    df.stop_time =  pd.to_datetime(df.stop_time, format='%m/%d/%Y %H:%M:%S', errors='ignore')
    df.stop_time =  pd.to_datetime(df.stop_time, format='%m/%d/%Y %H:%M', errors='ignore')
    df.stop_time =  pd.to_datetime(df.stop_time, format='%Y-%m-%d %H:%M:%S', errors='ignore')

In [5]:
# concatenate all dfs into a single master one
master_df = pd.concat(dfs, ignore_index=True)

In [7]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79402697 entries, 0 to 79402696
Data columns (total 15 columns):
trip_duration         int64
start_time            datetime64[ns]
stop_time             datetime64[ns]
start_station_id      float64
start_station_name    object
start_station_lat     float64
start_station_long    float64
end_station_id        float64
end_station_name      object
end_station_lat       float64
end_station_long      float64
bike_id               int64
user_type             object
birth_year            object
gender                int64
dtypes: datetime64[ns](2), float64(6), int64(3), object(4)
memory usage: 8.9+ GB


In [9]:
master_df.sample(n=10)

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_lat,start_station_long,end_station_id,end_station_name,end_station_lat,end_station_long,bike_id,user_type,birth_year,gender
60518498,373,2018-06-15 21:10:26.882,2018-06-15 21:16:39.923,509.0,9 Ave & W 22 St,40.745497,-74.001971,435.0,W 21 St & 6 Ave,40.741740,-73.994156,20729,Subscriber,1988,1
78879170,305,2017-11-01 18:48:58.000,2017-11-01 18:54:04.000,3272.0,Jersey & 3rd,40.723332,-74.045953,3275.0,Columbus Drive,40.718355,-74.038914,31704,Subscriber,1991,2
25412263,698,2016-03-19 22:17:09.000,2016-03-19 22:28:47.000,526.0,E 33 St & 5 Ave,40.747659,-73.984907,2003.0,1 Ave & E 18 St,40.733812,-73.980544,23200,Subscriber,1989,2
71969069,789,2019-01-17 17:19:45.609,2019-01-17 17:32:55.171,3443.0,W 52 St & 6 Ave,40.761330,-73.979820,490.0,8 Ave & W 33 St,40.751551,-73.993934,15424,Subscriber,1954,1
32451112,994,2016-09-05 15:50:19.000,2016-09-05 16:06:54.000,335.0,Washington Pl & Broadway,40.729039,-73.994046,473.0,Rivington St & Chrystie St,40.721101,-73.991925,26627,Subscriber,1958,1
61035961,158,2018-06-08 19:05:06.204,2018-06-08 19:07:44.444,3285.0,W 87 St & Amsterdam Ave,40.788390,-73.974700,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,25292,Subscriber,1998,1
76533588,474,2019-05-03 07:57:09.604,2019-05-03 08:05:04.537,545.0,E 23 St & 1 Ave,40.736502,-73.978095,3641.0,Broadway & W 25 St,40.742869,-73.989186,19245,Subscriber,1990,1
18382114,343,2015-08-06 18:48:06.000,2015-08-06 18:53:49.000,317.0,E 6 St & Avenue B,40.724537,-73.981854,300.0,Shevchenko Pl & E 7 St,40.728145,-73.990214,16565,Subscriber,1949,1
29949445,514,2016-07-15 09:08:30.000,2016-07-15 09:17:04.000,3151.0,E 81 St & York Ave,40.772838,-73.949892,3143.0,5 Ave & E 78 St,40.776829,-73.963888,21415,Subscriber,1979,2
35964329,1142,2016-11-10 14:53:51.000,2016-11-10 15:12:53.000,476.0,E 31 St & 3 Ave,40.743943,-73.979661,340.0,Madison St & Clinton St,40.712690,-73.987763,23978,Subscriber,1987,1


In [8]:
# change data types to more appropriate / efficient ones
# downcast trip duration
master_df.trip_duration = pd.to_numeric(master_df.trip_duration, downcast = 'unsigned')

In [10]:
# make birth_year column a float
master_df.birth_year = pd.to_numeric(master_df.birth_year, errors='coerce', downcast='float')

In [11]:
# downcast floats
float_cols = ['start_station_lat', 'start_station_long', 'end_station_lat', 'end_station_long']
for col in float_cols:
    master_df[col] = pd.to_numeric(master_df[col], downcast='float')

In [12]:
# convert appropriate columns to categories
category_cols = ['start_station_id', 'start_station_name', 'end_station_id', 'end_station_name', 'bike_id', 'user_type',
             'gender']
for col in category_cols:
    master_df[col] = master_df[col].astype('category')

In [13]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79402697 entries, 0 to 79402696
Data columns (total 15 columns):
trip_duration         uint32
start_time            datetime64[ns]
stop_time             datetime64[ns]
start_station_id      category
start_station_name    category
start_station_lat     float32
start_station_long    float32
end_station_id        category
end_station_name      category
end_station_lat       float32
end_station_long      float32
bike_id               category
user_type             category
birth_year            float32
gender                category
dtypes: category(7), datetime64[ns](2), float32(5), uint32(1)
memory usage: 3.8 GB


In [40]:
# replace numeric representation of gender for actual gender
master_df.gender = master_df.gender.replace({0: np.nan, 1: 'male', 2: 'female'}).astype('category')

In [41]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79402697 entries, 0 to 79402696
Data columns (total 15 columns):
trip_duration         uint32
start_time            datetime64[ns]
stop_time             datetime64[ns]
start_station_id      category
start_station_name    category
start_station_lat     float32
start_station_long    float32
end_station_id        category
end_station_name      category
end_station_lat       float32
end_station_long      float32
bike_id               category
user_type             category
birth_year            float32
gender                category
dtypes: category(7), datetime64[ns](2), float32(5), uint32(1)
memory usage: 3.8 GB


In [42]:
master_df.isna().sum()

trip_duration               0
start_time                  0
stop_time                   0
start_station_id         2534
start_station_name       2534
start_station_lat           0
start_station_long          0
end_station_id          20593
end_station_name        20593
end_station_lat         18059
end_station_long        18059
bike_id                     0
user_type               52277
birth_year            6273717
gender                8349248
dtype: int64

In [52]:
# Bronx rides
master_df.loc[master_df.start_station_id.isna()]

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_lat,start_station_long,end_station_id,end_station_name,end_station_lat,end_station_long,bike_id,user_type,birth_year,gender
64257903,2544,2018-08-15 11:31:37.867,2018-08-15 12:14:02.129,NaN,NaN,40.856998,-73.889999,NaN,NaN,40.856998,-73.889999,34426,Subscriber,1971.0,female
64258082,2004,2018-08-15 11:35:26.014,2018-08-15 12:08:50.036,NaN,NaN,40.856998,-73.889999,NaN,NaN,40.856998,-73.887001,34428,Subscriber,1976.0,female
64259819,213,2018-08-15 12:10:37.436,2018-08-15 12:14:11.292,NaN,NaN,40.856998,-73.889999,NaN,NaN,40.856998,-73.889999,34428,Subscriber,1976.0,female
64260053,602,2018-08-15 12:15:17.333,2018-08-15 12:25:19.762,NaN,NaN,40.845001,-73.902000,NaN,NaN,40.856998,-73.889999,34525,Subscriber,1988.0,male
64261416,511502,2018-08-15 12:41:41.969,2018-08-21 10:46:44.644,NaN,NaN,40.856998,-73.889999,NaN,NaN,40.856998,-73.889999,34548,Subscriber,1986.0,male
64262099,3471,2018-08-15 12:53:40.817,2018-08-15 13:51:32.009,NaN,NaN,40.856998,-73.889999,NaN,NaN,40.845001,-73.896004,34501,Subscriber,1993.0,male
64262155,3108,2018-08-15 12:54:48.029,2018-08-15 13:46:36.756,NaN,NaN,40.854000,-73.889999,NaN,NaN,40.845001,-73.892998,34518,Subscriber,1989.0,female
64265601,1738,2018-08-15 13:58:46.212,2018-08-15 14:27:45.173,NaN,NaN,40.845001,-73.892998,NaN,NaN,40.839001,-73.877998,34518,Subscriber,1989.0,female
64265661,644,2018-08-15 13:59:50.331,2018-08-15 14:10:34.866,NaN,NaN,40.845001,-73.892998,NaN,NaN,40.841999,-73.884003,34501,Subscriber,1993.0,male
64266796,443,2018-08-15 14:20:31.586,2018-08-15 14:27:54.944,NaN,NaN,40.841999,-73.884003,NaN,NaN,40.839001,-73.877998,34501,Subscriber,1993.0,male


In [58]:
master_df.loc[(master_df.end_station_id.isna()) & (master_df.start_station_id.notna())]

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_lat,start_station_long,end_station_id,end_station_name,end_station_lat,end_station_long,bike_id,user_type,birth_year,gender
56,547,2013-06-01 00:11:04,2013-06-01 00:20:11,432.0,E 7 St & Avenue A,40.726219,-73.983795,NaN,NaN,NaN,NaN,17470,Subscriber,1980.0,male
102,537,2013-06-01 00:40:27,2013-06-01 00:49:24,482.0,W 15 St & 7 Ave,40.739357,-73.999321,NaN,NaN,NaN,NaN,15090,Subscriber,1981.0,male
120,472,2013-06-01 00:47:51,2013-06-01 00:55:43,528.0,2 Ave & E 31 St,40.742908,-73.977058,NaN,NaN,NaN,NaN,16257,Subscriber,1965.0,male
211,153,2013-06-01 01:32:55,2013-06-01 01:35:28,284.0,Greenwich Ave & 8 Ave,40.739017,-74.002640,NaN,NaN,NaN,NaN,20106,Subscriber,1987.0,male
289,841,2013-06-01 02:28:10,2013-06-01 02:42:11,509.0,9 Ave & W 22 St,40.745499,-74.001968,NaN,NaN,NaN,NaN,18792,Customer,NaN,NaN
302,904,2013-06-01 02:40:31,2013-06-01 02:55:35,505.0,6 Ave & W 33 St,40.749012,-73.988487,NaN,NaN,NaN,NaN,15752,Customer,NaN,NaN
315,744,2013-06-01 02:54:35,2013-06-01 03:06:59,350.0,Clinton St & Grand St,40.715595,-73.987030,NaN,NaN,NaN,NaN,17106,Customer,NaN,NaN
334,914,2013-06-01 03:22:48,2013-06-01 03:38:02,233.0,Cadman Plaza W & Pierrepont St,40.694756,-73.990524,NaN,NaN,NaN,NaN,16461,Subscriber,1988.0,male
354,697,2013-06-01 03:52:23,2013-06-01 04:04:00,528.0,2 Ave & E 31 St,40.742908,-73.977058,NaN,NaN,NaN,NaN,17283,Customer,NaN,NaN
373,620,2013-06-01 04:50:53,2013-06-01 05:01:13,435.0,W 21 St & 6 Ave,40.741741,-73.994156,NaN,NaN,NaN,NaN,18327,Subscriber,1949.0,male
